In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *

In [2]:
world = SDFWorld()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
frame = Frame(world.vis, "frame")
frame_curr = Frame(world.vis, "frame_curr")

In [4]:
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)

In [5]:
panda = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda.reduce_dim([7, 8], [0.04, 0.04])

In [9]:
def get_ee_pose(q):
    return panda_model.fk_fn(q)[-1]
def get_qtn_map_mat(qtn):
    q0, q1, q2, q3 = qtn
    return jnp.array([
        [-q1, q0, -q3, q2],
        [-q2, q3, q0, -q1],
        [-q3, -q2, q1, q0],
    ])

In [253]:
qtn = np.random.normal(size=4)
qtn = qtn/np.linalg.norm(qtn)
target_pose = SE3.from_rotation_and_translation(
    SO3(qtn), jnp.array([0.4, 0.05, 0.4])).parameters()
frame.set_pose(SE3(target_pose))

In [254]:
#init
q = panda.neutral
panda.set_joint_angles(q)

In [255]:
def analytical_jacobian_qtn(q):
    ee_pose = panda_model.fk_fn(q)[-1]
    geom_jac = panda_model.jac_fn(q) #wrt world
    pos_jac = geom_jac[:3, :]
    rot_jac_angvel = geom_jac[3:, :]
    H = get_qtn_map_mat(ee_pose[:4])
    rot_jac_qtn = 1/2 * H.T @ rot_jac_angvel
    return jnp.vstack([pos_jac, rot_jac_qtn])

In [363]:
ee_pose = panda_model.fk_fn(q)[-1]
geom_jac = panda_model.jac_fn(q) #wrt world
pos_jac = geom_jac[:3, :]
rot_jac_angvel = geom_jac[3:, :]

pos_jac_body = SO3(ee_pose[:4]).as_matrix().T @ pos_jac
rot_jac_body = SO3(ee_pose[:4]).as_matrix().T @ rot_jac_angvel

qtn_target = (SO3(ee_pose[:4]).inverse() @ SO3(target_pose[:4])).parameters()
H = get_qtn_map_mat(qtn_target)
rot_jac_qtn = 1/2 * H.T @ rot_jac_body
jac = jnp.vstack([pos_jac_body, rot_jac_qtn[1:,:]])

pos_err = SO3(ee_pose[:4]).inverse().apply(target_pose[-3:] - ee_pose[-3:])
rot_err = qtn_target[1:]

error = jnp.hstack([pos_err, rot_err])
#jac_pinv = np.linalg.pinv(jac)
q_delta = jac.T @ error

q += q_delta*1.
panda.set_joint_angles(q)
print(rot_err)
print(ee_pose[0])

[ 0.03280118  0.00828883 -0.00603953]
0.8077729


In [46]:
error

Array([ 0.00928172,  0.00740224,  0.00683788, -0.03211983,  0.0250527 ,
       -0.02126297], dtype=float32)

In [45]:
jac.T

Array([[ 7.3535487e-02, -2.0712869e-01,  3.4838462e-01,  4.8920855e-01,
         8.0805145e-02, -6.3188925e-02],
       [ 3.9328170e-01,  2.1349944e-02, -9.4064571e-02, -1.0126683e-01,
         3.4390864e-01, -3.4799409e-01],
       [ 3.7498474e-02, -1.8973646e-01,  3.2346365e-01,  4.6685290e-01,
         1.7491509e-01,  3.6374912e-02],
       [-1.9736950e-01, -1.5633115e-01, -1.3805738e-01,  1.2512822e-01,
        -3.9369780e-01,  2.8098023e-01],
       [-9.9392697e-02, -1.8150987e-01,  1.0244548e-08, -2.7937528e-01,
         1.7814527e-01,  3.7441111e-01],
       [ 1.8594682e-01, -1.0182233e-01, -8.7999985e-02, -2.4921830e-01,
        -4.3298435e-01,  2.0102682e-02],
       [-7.9794003e-09,  1.7375532e-09, -3.1136788e-10, -1.2526370e-02,
        -1.6059941e-02, -4.9947190e-01]], dtype=float32)

In [3658]:
jac

Array([[-7.7002821e-03,  2.4585027e-01, -1.9977307e-02, -5.6794697e-01,
        -1.0608960e-02, -2.0810066e-01, -4.6566129e-10],
       [-2.4906400e-01,  6.8868876e-02, -2.0071512e-01,  1.3302273e-01,
         4.9682923e-02, -4.2795356e-02,  1.0710210e-08],
       [ 0.0000000e+00, -2.3775479e-01,  1.3434693e-02,  2.1414879e-01,
        -9.3831425e-04,  8.6894460e-02, -1.8917490e-10],
       [ 2.4423324e-01, -3.1565249e-03,  2.4088922e-01, -1.5847936e-02,
        -2.4149153e-01, -1.1343132e-03, -2.4423325e-01],
       [ 4.3626493e-01, -3.6058852e-03,  4.2856318e-01, -3.4465697e-02,
        -4.2976633e-01, -4.6560867e-03, -4.3626496e-01],
       [ 2.6207364e-03,  3.5421392e-01, -6.0970344e-02, -4.7458217e-01,
         5.8485121e-02, -3.7562874e-01,  2.6207366e-03]], dtype=float32)

In [3656]:
jnp.linalg.norm(ee_qtn)

Array(0.9999998, dtype=float32)

In [3338]:
rot_err

Array([-0.34707612,  0.8533501 , -0.38542277,  0.05273522], dtype=float32)

In [3335]:
jnp.linalg.cond(jac[:3, :])

Array(3.063508, dtype=float32)

In [3273]:
SE3(ee_pose).rotation().inverse()

SO3(wxyz=[-0.83207     0.03318     0.52836996  0.16544999])

In [3266]:
pos_jac, qtn_jac = analytical_jacobian_qtn(q)
ee_pose = get_ee_pose(q)

jac = jnp.vstack([pos_jac, qtn_jac[1:]])


pos_delta = target_pose[-3:] - ee_pose[-3:]
pos_mag = 2.
pos_delta = pos_delta if np.linalg.norm(pos_delta) < pos_mag else pos_delta/np.linalg.norm(pos_delta)*pos_mag

qtn_target = target_pose[:4]
rot_target_vec = SO3(qtn_target).log()
rot_angle = jnp.linalg.norm(rot_target_vec)
rot_delta = SO3(qtn_target) \
    if rot_angle < 3. \
    else SO3.exp(rot_target_vec/rot_angle*3.)

#rot_delta = SO3(target_pose[:4]) # since we are on the world frame
#(SO3(ee_pose[:4]).inverse() @ SO3(target_pose[:4]))
#rot_delta = SO3(ee_pose[:4])@rot_delta
# rot_delta_vec = rot_delta.log()
# rot_delta_mag = jnp.linalg.norm(rot_delta_vec)
# rot_delta = rot_delta \
#     if rot_delta_mag < 0.5 \
#     else SO3.exp(rot_delta_vec/rot_delta_mag*0.5)

qtn_delta = rot_delta.parameters()
qtn_delta = qtn_delta[1:] if qtn_delta[0] >= 0 else -qtn_delta[1:]

delta = jnp.hstack([pos_delta, qtn_delta])
#delta = jnp.hstack([pos_delta, jnp.zeros(3)])
#delta = jnp.hstack([jnp.zeros(3), qtn_delta])

q_vel = jnp.linalg.pinv(jac) @ delta

q = q + q_vel * 0.1
print(rot_delta)
panda.set_joint_angles(q)

SO3(wxyz=[ 0.14987999 -0.80438    -0.57364    -0.03809   ])


In [2742]:
qtn_delta = rot_delta.parameters()
qtn_delta

Array([ 0.9689124 , -0.22714303, -0.08552869, -0.04795405], dtype=float32)

In [2743]:
qtn_delta = qtn_delta[1:] if qtn_delta[0] >= 0 else -qtn_delta[1:]
qtn_delta

Array([-0.22714303, -0.08552869, -0.04795405], dtype=float32)

In [65]:
qtn_vel = (SO3(ee_pose[:4]).inverse()@SO3(target_pose[:4])).parameters()

In [26]:
q = panda.neutral

Array([[-0.06827764, -0.18924005, -0.06827764,  0.18924005, -0.4568656 ,
         0.18924008, -0.06827764],
       [-0.18924005,  0.06827764, -0.18924005, -0.06827766,  0.0282822 ,
        -0.06827763,  0.18924001],
       [ 0.4568659 ,  0.02828152,  0.4568659 , -0.0282815 , -0.06827927,
        -0.02828156, -0.45686582],
       [ 0.02828152, -0.4568659 ,  0.02828152,  0.4568659 ,  0.18923992,
         0.45686597,  0.02828153]], dtype=float32)

In [31]:
geom_jac

Array([[-4.6566129e-10,  2.2148973e-01, -4.6566129e-10,  9.4510257e-02,
         1.6653345e-15,  1.7700882e-01,  8.9062968e-10],
       [ 6.1263680e-01,  0.0000000e+00,  6.1263680e-01,  3.1598617e-08,
         1.7700827e-01, -1.1160222e-08,  7.4505806e-09],
       [ 0.0000000e+00, -6.1263680e-01,  0.0000000e+00,  5.3013682e-01,
         4.6566123e-10,  1.4613657e-01,  2.7228692e-10],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         9.9999988e-01,  0.0000000e+00,  2.9236585e-01],
       [ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00, -1.0000000e+00,
         0.0000000e+00, -1.0000001e+00,  0.0000000e+00],
       [ 1.0000000e+00,  0.0000000e+00,  1.0000000e+00,  5.9604645e-08,
        -3.5762787e-06, -7.6368451e-08, -9.5630634e-01]], dtype=float32)

(6, 7)

In [3]:
frame = Frame(world.vis, "frame")
frame_curr = Frame(world.vis, "frame_curr")

In [12]:
frame.set_pose(SE3.identity())

In [19]:
qtn = np.random.random(4)
qtn /= np.linalg.norm(qtn)
qtn_curr = qtn

In [20]:
frame_curr.set_pose(SE3.from_rotation(SO3(qtn_curr)))

In [23]:
qtn_target = jnp.array([1, 0,0,0.])

In [278]:
angvel = (SO3(qtn_curr).inverse() @ SO3(qtn_target)).log()
H = get_qtn_map_mat(qtn_curr)
gain = 0.1
qtn_update = gain * 1/2*H.T @ angvel

qtn_curr += qtn_update
frame_curr.set_pose(SE3.from_rotation(SO3(qtn_curr)))

In [40]:
skew(xyz)+jnp.eye(3)*w

Array([[ 0.31938317, -0.49195808,  0.39933908],
       [ 0.49195808,  0.31938317, -0.7046275 ],
       [-0.39933908,  0.7046275 ,  0.31938317]], dtype=float32)

In [45]:
H = get_qtn_map_mat(qtn_curr)
1/2 * H.T

Array([[-0.35231376, -0.19966954, -0.24597904],
       [ 0.15969159,  0.24597904, -0.19966954],
       [-0.24597904,  0.15969159,  0.35231376],
       [ 0.19966954, -0.35231376,  0.15969159]], dtype=float32)

In [36]:
skew(xyz) @ jnp.ones(3)

Array([-0.092619  , -0.21266943,  0.30528843], dtype=float32)

In [27]:
angvel

Array([-1.8525608, -1.0499163, -1.2934241], dtype=float32)